In [1]:
import numpy as np
from scipy.linalg import logm
import itertools

# Define omega = e^(2πi/3)
omega = np.exp(2j * np.pi / 3)

# Define the standard Gell-Mann matrices for SU(3)
def gell_mann_matrices():
    """Return the 8 Gell-Mann matrices as a list"""
    
    # λ₁ - symmetric off-diagonal
    lambda1 = np.array([[0, 1, 0],
                        [1, 0, 0],
                        [0, 0, 0]], dtype=complex)
    
    # λ₂ - antisymmetric off-diagonal  
    lambda2 = np.array([[0, -1j, 0],
                        [1j, 0, 0],
                        [0, 0, 0]], dtype=complex)
    
    # λ₃ - diagonal in computational subspace
    lambda3 = np.array([[1, 0, 0],
                        [0, -1, 0],
                        [0, 0, 0]], dtype=complex)
    
    # λ₄ - couples |0⟩ ↔ |2⟩
    lambda4 = np.array([[0, 0, 1],
                        [0, 0, 0],
                        [1, 0, 0]], dtype=complex)
    
    # λ₅ - couples |0⟩ ↔ |2⟩ (imaginary)
    lambda5 = np.array([[0, 0, -1j],
                        [0, 0, 0],
                        [1j, 0, 0]], dtype=complex)
    
    # λ₆ - couples |1⟩ ↔ |2⟩
    lambda6 = np.array([[0, 0, 0],
                        [0, 0, 1],
                        [0, 1, 0]], dtype=complex)
    
    # λ₇ - couples |1⟩ ↔ |2⟩ (imaginary)
    lambda7 = np.array([[0, 0, 0],
                        [0, 0, -1j],
                        [0, 1j, 0]], dtype=complex)
    
    # λ₈ - overall diagonal (hypercharge-like)
    lambda8 = (1/np.sqrt(3)) * np.array([[1, 0, 0],
                                         [0, 1, 0],
                                         [0, 0, -2]], dtype=complex)
    
    return [lambda1, lambda2, lambda3, lambda4, lambda5, lambda6, lambda7, lambda8]

def displacement_operators():
    """Return the 9 displacement operators D(p,q) = ω^(pq) Z^p X^q"""
    
    # Define Z (clock operator) and X (shift operator)
    Z = np.array([[1, 0, 0],
                  [0, omega, 0],
                  [0, 0, omega**2]], dtype=complex)
    
    X = np.array([[0, 0, 1],
                  [1, 0, 0],
                  [0, 1, 0]], dtype=complex)
    
    displacements = {}
    
    for p in range(3):
        for q in range(3):
            # D(p,q) = ω^(pq) Z^p X^q
            phase_factor = omega**(p*q)
            Z_power = np.linalg.matrix_power(Z, p)
            X_power = np.linalg.matrix_power(X, q)
            
            D_pq = phase_factor * Z_power @ X_power
            displacements[(p, q)] = D_pq
    
    return displacements

def conjugate_matrix(U, A):
    """Compute U A U†"""
    return U @ A @ U.conj().T

def decompose_into_gell_mann(matrix, gell_mann_list):
    """Decompose a 3x3 matrix into Gell-Mann basis coefficients"""
    coefficients = []
    
    for i, lambda_k in enumerate(gell_mann_list):
        # coefficient = (1/2) * Tr(matrix * λₖ)
        coeff = 0.5 * np.trace(matrix @ lambda_k)
        coefficients.append(coeff)
    
    return coefficients

def find_permutation_in_subspace(original_coeffs, conjugated_coeffs, leakage_indices):
    """Find if conjugated coefficients are a permutation of original within leakage subspace"""
    
    # Extract leakage components
    orig_leak = [original_coeffs[i] for i in leakage_indices]
    conj_leak = [conjugated_coeffs[i] for i in leakage_indices]
    
    # Check if it's a permutation (within numerical tolerance)
    tol = 1e-12
    
    # Sort both arrays and compare
    orig_sorted = sorted([abs(x) for x in orig_leak])
    conj_sorted = sorted([abs(x) for x in conj_leak])
    
    is_permutation = all(abs(o - c) < tol for o, c in zip(orig_sorted, conj_sorted))
    
    return is_permutation, orig_leak, conj_leak

def main():
    print("=== Qutrit Leakage Closure Property Verification ===\n")
    
    # Get Gell-Mann matrices and displacement operators
    gell_mann = gell_mann_matrices()
    displacements = displacement_operators()
    
    # Define leakage indices (λ₁, λ₂, λ₄, λ₅, λ₆, λ₇)
    leakage_indices = [0, 1, 3, 4, 5, 6]  # 0-indexed
    complementary_indices = [2, 7]  # λ₃, λ₈
    
    print(f"Leakage generators: λ{[i+1 for i in leakage_indices]}")
    print(f"Complementary generators: λ{[i+1 for i in complementary_indices]}")
    print()
    
    # Test closure for each displacement operator
    closure_violations = []
    
    for (p, q), D_pq in displacements.items():
        if (p, q) == (0, 0):  # Skip identity
            continue
            
        print(f"Testing displacement D({p},{q}):")
        
        # Test each leakage generator
        for leak_idx in leakage_indices:
            lambda_k = gell_mann[leak_idx]
            
            # Compute conjugation: D(p,q) λₖ D(p,q)†
            conjugated = conjugate_matrix(D_pq, lambda_k)
            
            # Decompose back into Gell-Mann basis
            original_coeffs = decompose_into_gell_mann(lambda_k, gell_mann)
            conjugated_coeffs = decompose_into_gell_mann(conjugated, gell_mann)
            
            # Check if non-leakage components remain zero
            for comp_idx in complementary_indices:
                if abs(conjugated_coeffs[comp_idx]) > 1e-12:
                    closure_violations.append(
                        f"D({p},{q}) λ{leak_idx+1} produces non-zero λ{comp_idx+1} component: {conjugated_coeffs[comp_idx]}"
                    )
        
        # Also test that complementary generators remain in their space
        for comp_idx in complementary_indices:
            lambda_k = gell_mann[comp_idx]
            conjugated = conjugate_matrix(D_pq, lambda_k)
            conjugated_coeffs = decompose_into_gell_mann(conjugated, gell_mann)
            
            # Check if leakage components remain zero
            for leak_idx in leakage_indices:
                if abs(conjugated_coeffs[leak_idx]) > 1e-12:
                    closure_violations.append(
                        f"D({p},{q}) λ{comp_idx+1} produces non-zero λ{leak_idx+1} component: {conjugated_coeffs[leak_idx]}"
                    )
    
    print("\n=== RESULTS ===")
    
    if closure_violations:
        print("❌ CLOSURE PROPERTY VIOLATED!")
        print("\nViolations found:")
        for violation in closure_violations:
            print(f"  • {violation}")
    else:
        print("✅ CLOSURE PROPERTY VERIFIED!")
        print("All leakage generators map to linear combinations of leakage generators only.")
        print("All complementary generators remain in the complementary subspace.")
    
    print(f"\nTotal displacement operators tested: {len(displacements) - 1}")
    print(f"Total conjugations computed: {(len(displacements) - 1) * 8}")
    
    # Additional verification: show explicit permutation tables for generators
    print("\n=== EXPLICIT PERMUTATION VERIFICATION ===")
    
    # Test the fundamental generators D(1,0) and D(0,1)
    test_displacements = [(1, 0), (0, 1)]
    
    for (p, q) in test_displacements:
        D_pq = displacements[(p, q)]
        print(f"\nD({p},{q}) conjugation effects on leakage generators:")
        
        leakage_mapping = []
        for leak_idx in leakage_indices:
            lambda_k = gell_mann[leak_idx]
            conjugated = conjugate_matrix(D_pq, lambda_k)
            
            # Find which leakage generator this becomes
            conjugated_coeffs = decompose_into_gell_mann(conjugated, gell_mann)
            
            # Find the dominant coefficient in leakage subspace
            max_coeff = 0
            dominant_idx = -1
            for i, idx in enumerate(leakage_indices):
                if abs(conjugated_coeffs[idx]) > max_coeff:
                    max_coeff = abs(conjugated_coeffs[idx])
                    dominant_idx = idx
            
            leakage_mapping.append(dominant_idx + 1)  # Convert to 1-indexed
        
        original_labels = [i+1 for i in leakage_indices]
        print(f"  λ{original_labels} → λ{leakage_mapping}")

if __name__ == "__main__":
    main()

=== Qutrit Leakage Closure Property Verification ===

Leakage generators: λ[1, 2, 4, 5, 6, 7]
Complementary generators: λ[3, 8]

Testing displacement D(0,1):
Testing displacement D(0,2):
Testing displacement D(1,0):
Testing displacement D(1,1):
Testing displacement D(1,2):
Testing displacement D(2,0):
Testing displacement D(2,1):
Testing displacement D(2,2):

=== RESULTS ===
✅ CLOSURE PROPERTY VERIFIED!
All leakage generators map to linear combinations of leakage generators only.
All complementary generators remain in the complementary subspace.

Total displacement operators tested: 8
Total conjugations computed: 64

=== EXPLICIT PERMUTATION VERIFICATION ===

D(1,0) conjugation effects on leakage generators:
  λ[1, 2, 4, 5, 6, 7] → λ[2, 1, 5, 4, 7, 6]

D(0,1) conjugation effects on leakage generators:
  λ[1, 2, 4, 5, 6, 7] → λ[6, 7, 1, 2, 4, 5]
